In [1]:
import pandas as pd
import numpy as np #storing as array
from sklearn import tree
from sklearn import preprocessing 
# used for finding missing values, text into numrical

In [2]:
train = pd.read_csv(r'C:\Users\HP\LetsUpgrade\Day-24\Dataset\HR_attrition_general_data.csv')
train.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [5]:
train['NumCompaniesWorked'] = np.where(train.NumCompaniesWorked.isnull(), train.NumCompaniesWorked.mean(), train.NumCompaniesWorked )
train['TotalWorkingYears'] = np.where(train.TotalWorkingYears.isnull(), train.TotalWorkingYears.mean(), train.TotalWorkingYears )

In [6]:
le = preprocessing.LabelEncoder()

In [7]:
train_temp = train.copy()

In [8]:
train_temp['Attrition'] = le.fit_transform(train.Attrition)
train_temp['BusinessTravel'] = le.fit_transform(train.BusinessTravel)
train_temp['Department'] = le.fit_transform(train.Department)
train_temp['EducationField'] = le.fit_transform(train.EducationField)
train_temp['Gender'] = le.fit_transform(train.Gender)
train_temp['JobRole'] = le.fit_transform(train.JobRole)
train_temp['MaritalStatus'] = le.fit_transform(train.MaritalStatus)
train_temp['Over18'] = le.fit_transform(train.Over18)

In [10]:
#inducing model, using randomforest classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 2000, max_features = 2, oob_score = True)

In [11]:
attr_train_feats = ['Age', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 
                'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 
                'Over18', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 
                'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [14]:
model.fit(X = train_temp[attr_train_feats], y = train_temp.Attrition)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
model.oob_score_

1.0

In [17]:
for train_feat, imp in zip(attr_train_feats, model.feature_importances_):
    print(train_feat, '-->', imp)

Age --> 0.09668957526357927
BusinessTravel --> 0.027895913757089028
Department --> 0.026513119915473126
DistanceFromHome --> 0.06978369988529742
Education --> 0.0407602250410899
EducationField --> 0.041475342074592
Gender --> 0.01821110037576959
JobLevel --> 0.037376740582267226
JobRole --> 0.05622786429631438
MaritalStatus --> 0.03974863813218266
MonthlyIncome --> 0.09415648336184347
NumCompaniesWorked --> 0.05658001722266492
Over18 --> 0.0
PercentSalaryHike --> 0.0655983622770788
StockOptionLevel --> 0.033774200013306305
TotalWorkingYears --> 0.0849088858095857
TrainingTimesLastYear --> 0.04510686169458108
YearsAtCompany --> 0.06862615946584555
YearsSinceLastPromotion --> 0.04299017099917282
YearsWithCurrManager --> 0.05357663983226687


#take away point is: 

Age, MonthlyIncome, TotalWorkingYears, DistanceFromHome, PercentSalaryHike, YearsAtCompany seems to be most importance among other features.


In [18]:
attr_test_feat = ['Age','DistanceFromHome', 'MonthlyIncome', 'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany']

In [19]:
model2 = tree.DecisionTreeClassifier(max_depth = 12) #max depth is : 6 varibale X 2 independt varibale =6x2: 12

In [21]:

model2.fit(X = train_temp[attr_test_feat], y = train_temp.Attrition)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=12,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [22]:
model2.score(X = train_temp[attr_test_feat], y = train_temp.Attrition)

0.9480725623582766

#  Age, MonthlyIncome, TotalWorkingYears, DistanceFromHome, PercentSalaryHike, YearsAtCompany features play important role in attrition level in the company, contributing 94.80%